# Instructions to execute code and add lessons to your calendar
* Do ABSOLUTELY step 1, others don't matter https://developers.google.com/calendar/quickstart/python
* Place the file credentials.json in the same directory where this notebook is
* Execute the first two cells and give authorization to your account
* Execute the *check()* function to be sure everything's working
* Change any parameter needed to set your options 
* Execute the *insert_events()* function with your parameters





In [ ]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import requests, json

In [ ]:

### AUTHENTICATION
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar']

creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('/pathto/token.pickle'):
    with open('/pathto/token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        # CHANGE WITH YOUR FILE PATH
        flow = InstalledAppFlow.from_client_secrets_file(
            '/pathto/credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    # CHANGE WITH YOUR FILE PATH
    with open('/pathto/token.pickle', 'wb') as token:
        pickle.dump(creds, token)
        
service = build('calendar', 'v3', credentials=creds)

In [ ]:
# Call the Calendar API
# prints out the list of your calendars with their id
def check():
    page_token = None
    calendar_list = service.calendarList().list(pageToken=page_token).execute()
    for calendar_list_entry in calendar_list['items']:
        print(calendar_list_entry['id'], calendar_list_entry['summary'])
check()

In [ ]:
# EVENTS INSERT
def insert_events(url, fromDate, toDate, notExams=[]):
    response = json.loads(requests.get(url).text)

    i = 0
    fromDate = datetime.datetime.strptime(fromDate, "%d-%m-%Y")
    toDate = datetime.datetime.strptime(toDate, "%d-%m-%Y")
    eventDate = datetime.datetime.strptime(response[i]['start'], "%Y-%m-%dT%H:%M:%S")
    while eventDate < fromDate :
        i += 1
        eventDate = datetime.datetime.strptime(response[i]['start'], "%Y-%m-%dT%H:%M:%S")
    
    while eventDate <= toDate:
        jEvent = response[i]
        if jEvent['cod_modulo'] not in notExams:
            location = ''
            desc = ''
            if(len(jEvent['aule']) > 0):
                location = jEvent['aule'][0]['des_indirizzo'].replace(' -', ',')
                for a in jEvent['aule']:
                    desc += a['des_risorsa'] + ', ' + a['des_piano'] + ' - ' + a['des_ubicazione'] + '\n'
            desc += 'Professor: ' + jEvent['docente']
            if type(jEvent['teams']) is str:
                desc += '\nTeams: ' + jEvent['teams'] + '\n'
            event = {
            'summary': jEvent['cod_modulo'] + ' - ' + jEvent['title'],
            'location': location,
            'description': desc ,
            'start': {
                'dateTime': jEvent['start'],
                'timeZone': 'Europe/Rome',
            },
            'end': {
                'dateTime': jEvent['end'],
                'timeZone': 'Europe/Rome',
            },
            'reminders': {
                'useDefault': False,
                'overrides': [
                {'method': 'popup', 'minutes': 60},
                ],
            },
            }
            # if you want to add it to your primary calendar just use calendarId="primary"
            event = service.events().insert(calendarId='primary', body=event).execute()
            print('Event created succesfully : %s' % (event.get('htmlLink')))
        i += 1
        eventDate = datetime.datetime.strptime(response[i]['start'], "%Y-%m-%dT%H:%M:%S")

In [ ]:
# PARAMETER url
first_year = "https://corsi.unibo.it/2cycle/artificial-intelligence/timetable/@@orario_reale_json?"
second_year = "https://corsi.unibo.it/2cycle/artificial-intelligence/timetable/@@orario_reale_json?anno=2&curricula="
# PARAMETER fromDate
#   the initial Date
# PARAMETER toDate
#   the final date (excluded)
# PARAMETER notExams
#   a list of the exams that you don't want to be added coded as strings 
insert_events(second_year,
              '20-09-2021',
              '26-09-2021',
              ['91264'])